In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199774 sha256=2695911992e00d26111ae2887b4ab1ff4edd7f97fb47bc7e8dd624df34ce7499
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import fasttext

In [ ]:
model_en = fasttext.load_model("/content/drive/MyDrive/cc.en.300.bin")
model_en

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/ecommerceDataset.csv",names=["category","description"],header=None)
print(df.shape)
df.head()

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [ ]:
df.category.value_counts()

Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: category, dtype: int64

In [ ]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [ ]:
# replacinng Clothing & Accessories with Clothing_Accessories
df.category.replace("Clothing & Accessories","Clothing_Accessories",inplace=True)
df.category.value_counts()

Household               19313
Books                   11820
Electronics             10621
Clothing_Accessories     8670
Name: category, dtype: int64

In [ ]:
# for fasttext model traning file should be in this format
# __label__household Paper Plane Design Framed Wall Hanging

In [ ]:
# add __label__ prefix to all categories
df["category"] = "__label__"+df["category"].astype(str)
df.head()

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [ ]:
df["category_description"] = df["category"]+" "+df["description"]
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


In [ ]:
df.category_description[0]

'__label__Household Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and so

In [ ]:
import re
text = "  This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours."
        # subtitue which is not word and not space with space
text = re.sub(r'[^\w\s]',' ',text)
text = re.sub(r'\s{2,}',' ',text)
text = text.strip().lower()
text

'this painting is ready to hang you would be proud to possess this unique painting that is a niche apart we use only the most modern and efficient printing technology on our prints with only the and inks and precision epson roland and hp printers this innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime we print solely with top notch 100 inks to achieve brilliant and true colours'

In [ ]:
def preprocess(text):
  # replace punct
  text = re.sub(r'[^\w\s]',' ',text)
  # replace multiple white spaces
  text = re.sub(r'\s{2,}',' ',text)
  # remove leading space and lowercase them
  return text.strip().lower()

In [ ]:
df["category_description"] = df["category_description"].map(preprocess)
df.head(3)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf floral framed painting ...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf uv textured modern art ...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train,test = train_test_split(df,test_size=0.2)
print(train.shape,test.shape)

(40339, 3) (10085, 3)


In [ ]:
# exporting only category_description col to csv file
train.to_csv("ecom_train",columns=["category_description"],index=False,header=False)
train.to_csv("ecom_test",columns=["category_description"],index=False,header=False)

In [ ]:
model = fasttext.train_supervised(input = "/content/ecom_train")
model.test("/content/ecom_test")   #size precision recall

(40339, 0.9825726963980267, 0.9825726963980267)

In [ ]:
model.predict("This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours.")

(('__label__household',), array([0.87760192]))